In [1]:
import numpy as np
import pymorphy2
import string
import re
import os
import pandas as pd
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords

In [2]:
# import nltk
# nltk.download('stopwords')
# C:\Users\admin\AppData\Roaming\nltk_data

In [3]:
# ds = pd.read_excel('d:/Projects/MegaJack/data/Таблица обращений итоговая_Kostya.xlsx')
# ds = ds.dropna()
# ds['ИД'] = ds['ИД'].astype('int16')

ds = pd.read_excel(r'd:\Projects\MegaJack\rosatom\data\Класс карточек\Класс карточек.xlsx')
ds = ds.dropna()

In [4]:
ds.shape[0]

31

In [5]:
ds.head()

,Класс,id,Текст
0,Постоянные заявки,1,Ежедневно проводить уборку помещения
1,Постоянные заявки,1,Каждые полдня измерять температуру
2,Постоянные заявки,1,Ежедневный отчёт о выполненных работах
3,Постоянные заявки,1,Ежедневный отчёт о невыполненных работах
4,Постоянные заявки,1,Ежемесячное предложение по улучшению работ


In [6]:
charsforexcluding = string.punctuation + u'«»№•–’‘”“\n\t¬…—'
global_morph = pymorphy2.MorphAnalyzer()
stop_words = [x for x in stopwords.words('english') + stopwords.words('russian') if not x in ["не"]]
stop_words += ["коллега","просить","просьба","здравствовать","спасибо","пожалуйста","уважаемый", "уважение"]
stop_words = set(stop_words)

def get_unigramms(decision, internal_morph=None):

    if isinstance(decision, str):
        morph = global_morph if internal_morph is None else internal_morph
        text = " " + decision.lower() + " "
        text = text.replace("добрый день", "").replace("доброе утро", "")

        unigramms = ""
        # заменим пунктуацию на пробел
        for x in charsforexcluding:
            text = text.replace(x, ' ')

        for el in text.split():

            el = el.lower()
            el = el.replace(" ", "")

            if (not el.isdigit()) & (not el in stop_words):
                if el != '':

                    prs = morph.parse(el)[0]
                    nf = prs.normal_form

                    if nf not in stop_words:
                        if unigramms == " ":
                            unigramms = nf
                        else:
                            unigramms += " "
                            unigramms += nf

        return unigramms

    else:
        return ""

In [7]:
def create_unigramms_in_ds(data_source, internal_morph=None):
    
    data_source['unigramms'] = data_source['Текст'].apply(lambda text: get_unigramms(text, internal_morph))
    data_source = data_source[~data_source['unigramms'].isnull()]

    return data_source

In [8]:
ds = create_unigramms_in_ds(ds)

In [9]:
ds

,Класс,id,Текст,unigramms
0,Постоянные заявки,1,Ежедневно проводить уборку помещения,ежедневно проводить уборка помещение
1,Постоянные заявки,1,Каждые полдня измерять температуру,каждый полдень измерять температура
2,Постоянные заявки,1,Ежедневный отчёт о выполненных работах,ежедневный отчёт выполнить работа
3,Постоянные заявки,1,Ежедневный отчёт о невыполненных работах,ежедневный отчёт невыполненный работа
4,Постоянные заявки,1,Ежемесячное предложение по улучшению работ,ежемесячный предложение улучшение работа
5,Постоянные заявки,1,Еженедельное совещание у генерального директора,еженедельный совещание генеральный директор
6,Постоянные заявки,1,Отчёт о работе за месяц,отчёт работа месяц
7,Постоянные заявки,1,План работ на месяц,план работа месяц
8,Постоянные заявки,1,Сдача отчётности в подразделения,сдача отчётность подразделение
9,Постоянные заявки,1,Ежемесячное обслуживание оборудования,ежемесячный обслуживание оборудование


### Делим выборку

In [10]:
CLASS_FILED = 'id'
# CLASS_FILED = 'Класс'

In [11]:
'Кол-во коассов: {}'.format(len(ds[CLASS_FILED].unique()))

'Кол-во коассов: 3'

In [12]:
train, test = train_test_split(ds[CLASS_FILED], random_state=1)

In [13]:
ds_train = ds.loc[train.index]
ds_test = ds.loc[test.index]

In [14]:
count_features = 3000
vect = TfidfVectorizer(sublinear_tf=True, use_idf=True, ngram_range=(1, 2), max_features=count_features)
features_train = vect.fit_transform(ds_train['unigramms'])
features_test = vect.transform(ds_test['unigramms'])

In [15]:
model = LogisticRegression(C=5, random_state=1, solver='liblinear', multi_class='ovr')
model.fit(features_train, ds_train[CLASS_FILED])

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=1,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [16]:
acc_train = metrics.accuracy_score(ds_train[CLASS_FILED], model.predict(features_train))
acc_test = metrics.accuracy_score(ds_test[CLASS_FILED], model.predict(features_test))

print('acc_train', acc_train)
print('acc_test', acc_test)

acc_train 1.0
acc_test 0.25


In [17]:
# чистый пердикт
# ds_test['pred'] = model.predict(features_test)

In [18]:
# предикт с вероятностями
probas = model.predict_proba(features_test)
class_indexes = np.argmax(probas, axis=1)  # индексы классов с max вероятностью
class_probas = probas[np.arange(features_test.shape[0]), class_indexes]
class_output = model.classes_[class_indexes]
ds_test[f'{CLASS_FILED} prob'] = class_probas
ds_test[f'{CLASS_FILED} pred'] = class_output
ds_test[f'{CLASS_FILED} success'] = (ds_test[f'{CLASS_FILED} pred'] == ds_test[CLASS_FILED]).astype('int16')

In [19]:
def create_conf_report(ds: pd.DataFrame):
    col_name = CLASS_FILED
    prob_col = f'{CLASS_FILED} prob'
    success_col = f'{CLASS_FILED} success'

    ds_rep = ds.sort_values(by=[prob_col], ascending=[False])
    ds_rep['{} Кол-во'.format(col_name)] = range(1, len(ds_rep) + 1)
    ds_rep['{} Соотв-сум'.format(col_name)] = ds_rep[success_col].cumsum()
    ds_rep['{} Точность'.format(col_name)] = ds_rep['{} Соотв-сум'.format(col_name)] / ds_rep['{} Кол-во'.format(col_name)]
    ds_rep['{} Доля'.format(col_name)] = ds_rep['{} Кол-во'.format(col_name)] / len(ds_rep)

    return ds_rep

In [20]:
ds_test = create_conf_report(ds_test)

In [21]:
ds_test.to_excel(r'd:\Projects\MegaJack\rosatom\data\Класс карточек\Класс карточек - Результат обучения - {}.xlsx'.format(CLASS_FILED))

### Обучаем на всех данных

In [22]:
model = LogisticRegression(C=5, random_state=1, solver='liblinear', multi_class='ovr')
features_all = vect.fit_transform(ds['unigramms'])
model.fit(features_all, ds[CLASS_FILED])

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2', random_state=1,
                   solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [23]:
pickle.dump(vect, open(r'd:\Projects\MegaJack\rosatom\data\Класс карточек\vect.pickle', 'wb'))
pickle.dump(model, open(r'd:\Projects\MegaJack\rosatom\data\Класс карточек\model.pickle', 'wb'))

### Тестируем

In [24]:
text = 'предоставить отчет'
unigramms = get_unigramms(text)
unigramms_vect = vect.transform([unigramms])
result = model.predict(unigramms_vect)

In [25]:
result[0]

1

In [26]:
result = model.predict_proba(unigramms_vect)
result

array([[0.56653533, 0.18734043, 0.24612424]])